In [14]:
%cd /scratch/edk202/ngram-prep

%env LC_ALL=C.UTF-8
%env LANG=C.UTF-8

%pip install -e . --no-build-isolation -q

/scratch/edk202/ngram-prep
env: LC_ALL=C.UTF-8
env: LANG=C.UTF-8
Note: you may need to restart the kernel to use updated packages.


In [6]:
# Auto-reload packages
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# Standard stuff
from pathlib import Path

# NLTK stuff
from nltk.corpus import stopwords; stopwords = set(stopwords.words("english"))
from nltk.stem import WordNetLemmatizer; lemmatizer = WordNetLemmatizer()

# Raw n-gram acquisition stuff
from ngram_acquire.pipeline.orchestrate import download_and_ingest_to_rocksdb
from ngram_acquire.pipeline.logger import setup_logger

# Cython utilities
from ngram_filter.config import PipelineConfig, FilterConfig
from ngram_filter.pipeline.orchestrator import build_processed_db

In [8]:
setup_logger(
    db_path="/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/1grams.db",
    console=False,
    rotate=True,
    max_bytes=100_000_000,
    backup_count=5,
    force=True
)

PosixPath('/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/ngram_download_20250917_080057.log')

## Download Unigrams and Ingest to a RocksDB Database

In [9]:
download_and_ingest_to_rocksdb(
    ngram_size = 1,
    repo_release_id = "20200217",
    repo_corpus_id = "eng",
    db_path = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/1grams.db",
    file_range = (0, 23),
    random_seed = 21,
    workers = 25,
    use_threads = False,
    ngram_type = "tagged",
    overwrite = True,
    write_batch_size = 100_000,
    open_type = "write:packed24",
    post_compact = True
)

Start Time: 2025-09-17 08:01:01

Download & Ingestion Configuration
Ngram repository:           https://storage.googleapis.com/books/ngrams/books/20200217/eng/eng-1-ngrams_exports.html
RocksDB database path:      /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/1grams.db
File index range:           0 to 23 (count ~ 24)
Total files available:      24
Files to process:           24
First file URL:             http://storage.googleapis.com/books/ngrams/books/20200217/eng/1-00012-of-00024.gz
Last file URL:              http://storage.googleapis.com/books/ngrams/books/20200217/eng/1-00005-of-00024.gz
Ngram size:                 1
Ngram filtering:            tagged
Overwrite mode:             True
Write batch size:           100,000
Worker processes/threads:   25 (processes)



Processing Files: 100%|██████████| 24/24 [07:58<00:00, 19.92s/files]



Starting post-ingestion compaction...
Compaction completed in 0:04:30.599447

Processing completed!
Fully processed files: 24
Total entries written: 41,783,218
Write batches flushed: 24
Uncompressed data processed: 43.28 GB
Processing throughput: 58.24 MB/sec

End Time: 2025-09-17 08:13:42.547605
Total Runtime: 0:12:40.916929

Time per file: 0:00:31.704872
Files per hour: 113.5


## Run Processing Pipeline

In [10]:
src_db = Path("/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/1grams.db")
dst_db = src_db.parent / "1grams_processed.db"
tmp_dir = src_db.parent / "processing_tmp"

# Default configs or override as desired
pipeline_config = PipelineConfig(
    src_db=src_db,
    dst_db=dst_db,
    tmp_dir=tmp_dir,
    readers=8,
    work_units_per_reader=16,
    force_restart=True,
    progress_every_s=60.0,
    output_whitelist_path=dst_db / "whitelist.txt",
    output_whitelist_top_n=60_000
)

filter_config = FilterConfig(
    stop_set=stopwords,
    lemma_gen=lemmatizer,
)

build_processed_db(pipeline_config, filter_config)

N-GRAM FILTER PIPELINE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Configuration:
════════════════════════════════════════════════════════════════════════════════════════════════════
  Workers: 8
  Work units: 128
  Source: /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/1grams.db
  Destination: 02/NLP_corpora/Google_Books/20200217/eng/5gram_files/1grams_processed.db
  Buffer: 25,000 items, 16MB
  Profile: write:packed24
  Input whitelist: None
  Output whitelist: .../20200217/eng/5gram_files/1grams_processed.db/whitelist.txt (top 60,000 keys)

Phase 1: Creating work units...
════════════════════════════════════════════════════════════════════════════════════════════════════
  Force restart requested - clearing existing work units
  Creating 128 work units using ASCII range...
  Created 128 work units covering range 0x21-0x7e

Phase 2: Processing 128 work units with 8 workers...
═══════════════════════════════════

In [12]:
from common_db.api import open_db

db_path = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/1grams_processed.db"

with open_db(db_path, mode="rw") as db:
    db.compact_all()